![NASA](http://www.nasa.gov/sites/all/themes/custom/nasatwo/images/nasa-logo.svg)
![DEVELOP](../../DEVELOP_logo.png)

---

# GIS Automation through ArcPy

### Goddard Space Flight Center

#### March 6, 2017

---

---

# ArcPy

---

* Python module created by Esri to allow ArcGIS tools to be called from Python
* This enables all manner of geoprocessing to be done in an automated fashion, as ArcPy can replicate most of the functionality of ArcGIS for Desktop
* While ArcGIS comes with its own version of Python, ArcPy can be used with other Python installations, as we will do in this course
* For reference, see the documentation for your version of ArcGIS. The help page for each tool has a description of the Python syntax and (usually) short example scripts.
    * For instance: http://desktop.arcgis.com/en/arcmap/10.4/tools/data-management-toolbox/clip.htm (scroll down)
* Another resource: [Penn State online course](https://www.e-education.psu.edu/geog485/node/91) covering the same subject matter as this lecture, GIS automation with Python.
* Possibly the best resource of all: Google
    * While not as well supported as the really popular Python packages (i.e. NumPy, matplotlib, etc.), enough people use ArcPy that you will find decent support on forums like Stack Overflow

# Example Project: Fun with MODIS LST Data

---

* To get a feel for automating typical GIS tasks, we will work through a short project where we:
    * download some MODIS land surface temperature (LST) data
    * extract the bands we want to GeoTIFF
    * perform some analysis, including raster math and conditionals
    * resample the output
    * ... and repeat

### Test ArcPy

In [ ]:
import arcpy

### Download data

---

Download this MODIS image: ftp://ladsweb.nascom.nasa.gov/allData/6/MOD11A2/2016/201/MOD11A2.A2016201.h11v05.006.2016242234243.hdf 

Copy it to the directory where you want to store your data.

### Set working directory

---

ArcPy needs to know where to look for inputs and save outputs on your computer. You can specify an absolute path for every file, meaning a path including not only the file name but all the parent directories, including the drive name (i.e. C:\\Users\\abhubba1\\Documents\\Python Scripts\\DEVELOP_class\\thatwasalotoftyping.tif). However, this can grow tedious, so it is usually easier to set the workspace environment (more on these later) to a location of your choosing:

In [ ]:
arcpy.env.workspace = "C:\\Users\\abhubba1\\Documents\\Python Scripts\\DEVELOP_class"

Change this path to the directory where you saved the MODIS data in the above step. Now, all you need to provide is a file name, and ArcPy will automatically go to this folder.

Another consideration is that, by default, ArcPy will not permit you to overwrite files that already exist. When you are developing a script, this is usually not what you want (but not always - be careful!). Allow ArcPy to overwrite output as follows:

In [ ]:
arcpy.env.overwriteOutput = True

### Extract from HDF to GeoTIFF

---

With many data types, we would be ready to jump straight in to geoprocessing. However, with MODIS data we have an immediate hurdle: HDF. Fully-featured handling of HDF files requires special Python packages, like h5py, as covered in the previous lecture. For our project, however, this would be overkill. Instead, we can use ArcPy's Extract SubDataset to pull out the data we want into a GeoTIFF.

Let's extract the daytime LST data, which is subdataset 0:

In [ ]:
MODIS_file = "MOD11A2.A2016201.h11v05.006.2016242234243.hdf"
day_LST = MODIS_file.rstrip(".hdf") + "_dayLST.tif"
arcpy.ExtractSubDataset_management(MODIS_file, day_LST, subdataset_index=0)

To inspect the output, open ArcMap and load this file. Add an imagery basemap as well and compare the two, noting areas of high and low temperature values. Do you see anything strange?

### Perform analysis

---

#### Raster Math

The above data has been scaled to allow it to be stored in an integer format, but this makes it difficult to interpret. To convert it back to Kelvin, we must apply the scale factor, which in this case is 0.02. To do this, we will perform some raster math.

In [ ]:
arcpy.CheckOutExtension("spatial")

scale = 0.02
day_LST_sc = arcpy.Raster(day_LST) * scale

arcpy.CheckInExtension("spatial")

ArcPy allows basic math operations to be applied on a pixel-by-pixel basis with Python syntax, but it is necessary to create an ArcPy Raster object from your raster file on disk. The math operation returns another ArcPy Raster object, which we have assigned to __day_LST_sc__. Think of these Raster objects in an analogous manner to the data types we covered before, like string, float, int, etc. They can be used in other math operations and passed to some (but not all) ArcGIS tools that take raster input.

Notice the lines before and after the middle block. Raster math requires the Spatial Analyst extension, so it is necessary to check out a license for this extension first. Checking the license back in at the end is not necessary for the code to function, but it is good practice, because your organization may have a license server with limited keys.

At the moment, our scaled daytime LST raster only exists in memory - we have not saved it to disk. To do so:

In [ ]:
path = MODIS_file.rstrip(".hdf") + "_scale.tif"
day_LST_sc.save(path)

#### Math with multiple rasters

In a similar manner, two or more rasters can be used in a formula using simple Python syntax. But first, we need two rasters! Let's repeat the above steps with the nighttime LST band (subdataset 4), from the same MODIS file:

In [ ]:
arcpy.CheckOutExtension("spatial")

MODIS_file = "MOD11A2.A2016201.h11v05.006.2016242234243.hdf"
night_LST = MODIS_file.rstrip(".hdf") + "_nightLST.tif"
arcpy.ExtractSubDataset_management(MODIS_file, night_LST, subdataset_index=4)

scale = 0.02
night_LST_sc = arcpy.Raster(night_LST) * scale

arcpy.CheckInExtension("spatial")

_Note:_ In this case, we did not save the scaled raster to disk. This is not actually necessary to use it in later operations, and not saving intermediates is a good way to conserve disk space.

Now, let's compute the difference between the daytime and nighttime LST:

In [ ]:
arcpy.CheckOutExtension("spatial")

diff = day_LST_sc - night_LST_sc
diff.save(MODIS_file.rstrip(".hdf")+"_diff.tif")

arcpy.CheckInExtension("spatial")

This will create a new Raster object where each pixel corresponds to the difference between the two old rasters. Open this new raster in ArcMap and compare it to a basemap.

#### Conditionals

Suppose we want to know where the above difference exceeds a certain threshold. We can answer this question using the Con tool, available with the Spatial Analyst extension.

In [ ]:
arcpy.CheckOutExtension("spatial")

thres = 15
diff_thres = arcpy.sa.Con(diff > thres, 1, 0)
diff_thres.save(MODIS_file.rstrip(".hdf")+"_thres.tif")

arcpy.CheckInExtension("spatial")

The middle block of code will create and save a raster that contains a 1 wherever the day-night difference exceeds 15, and a 0 wherever it does not.

### Resample

---

We now have a nice Boolean raster that we could use as a mask in some larger project. However, what if the other members of our project are working with the MODIS red and NIR bands, which are 250 m resolution? We can be helpful, and resample our raster to this resolution.

In [ ]:
out_cellsize = 231.6563583
res = MODIS_file.rstrip(".hdf") + "_res.tif"
arcpy.Resample_management(diff_thres, res, cell_size=out_cellsize)

We could have performed this resampling at any point in our processing chain, but we chose to do so at the end. Why might this be best?

### Repeat as necessary

---

We have output (hopefully), but only for one raster file, and when has a project ever involved only one raster file?? The efficiency of scripting ArcGIS in Python becomes apparent when one must do the same thing many times, which should sound like something you have already learned... loops!

First, we'll need some more data. Since we already have a summer image, let's download files from:
* winter: ftp://ladsweb.nascom.nasa.gov/allData/6/MOD11A2/2016/017/MOD11A2.A2016017.h11v05.006.2016234002041.hdf
* spring: ftp://ladsweb.nascom.nasa.gov/allData/6/MOD11A2/2016/105/MOD11A2.A2016105.h11v05.006.2016242152502.hdf
* and fall: ftp://ladsweb.nascom.nasa.gov/allData/6/MOD11A2/2016/289/MOD11A2.A2016289.h11v05.006.2016302010943.hdf

Move these to the same folder you have been using.

Now, let's put everything we did above in a loop:

In [ ]:
import arcpy

arcpy.env.workspace = "C:\\Users\\abhubba1\\Documents\\Python Scripts\\DEVELOP_class"
arcpy.env.overwriteOutput = True
arcpy.CheckOutExtension("spatial")

scale = 0.02
thres = 15
out_cellsize = 231.6563583
MODIS_files = ["MOD11A2.A2016017.h11v05.006.2016234002041.hdf", 
               "MOD11A2.A2016105.h11v05.006.2016242152502.hdf", 
               "MOD11A2.A2016201.h11v05.006.2016242234243.hdf", 
               "MOD11A2.A2016289.h11v05.006.2016302010943.hdf"]

for f in MODIS_files:
    
    #Extract from HDF
    day_LST = f.rstrip(".hdf") + "_dayLST.tif"
    arcpy.ExtractSubDataset_management(f, day_LST, subdataset_index=0)
    night_LST = f.rstrip(".hdf") + "_nightLST.tif"
    arcpy.ExtractSubDataset_management(f, night_LST, subdataset_index=4)
    
    #Scale
    day_LST_sc = arcpy.Raster(day_LST) * scale
    night_LST_sc = arcpy.Raster(night_LST) * scale
    
    #Perform subtraction
    diff = day_LST_sc - night_LST_sc
    diff.save(f.rstrip(".hdf")+"_diff.tif")
    
    #Apply conditional
    diff_thres = arcpy.sa.Con(diff > thres, 1, 0)
    diff_thres.save(f.rstrip(".hdf")+"_thres.tif")
    
    #Resample
    res = f.rstrip(".hdf") + "_res.tif"
    arcpy.Resample_management(diff_thres, res, cell_size=out_cellsize)
    
arcpy.CheckInExtension("spatial")

_Tip:_ If you need to get the names of many files in a folder and don't want to do a lot of tedious typing, check out the  [glob](https://docs.python.org/2/library/glob.html) module.



# Other Considerations

---

### Environments

---

ArcGIS uses environment settings to control the specific characteristics of your current geoprocessing environment. In the above script, we are using two already: workspace and overwrite output. Environments are a powerful way to control the exact way in which your script executes. As another example, let's say that within our MODIS tile, we only really care about Virginia (because it is objectively better than all of these other states), and we want to limit output to this state.

One approach might be to clip each input raster with a shapefile, but we have to do this for each input. In this case, we only have two per iteration, but in other projects we might have a lot more. A more elegant approach is to use the mask environment setting.

Download [this](https://drive.google.com/file/d/0B8toI67HoVSrVzV3aUcwc3FBcG8/view?usp=sharing) shapefile of Virginia, and extract it to your work directory. Then, rerun our script with the mask environment set at the top:

In [ ]:
import arcpy

arcpy.env.workspace = "C:\\Users\\abhubba1\\Documents\\Python Scripts\\DEVELOP_class"
arcpy.env.overwriteOutput = True
arcpy.env.mask = "Virginia.shp"
arcpy.CheckOutExtension("spatial")

scale = 0.02
thres = 15
out_cellsize = 231.6563583
MODIS_files = ["MOD11A2.A2016017.h11v05.006.2016234002041.hdf", 
               "MOD11A2.A2016105.h11v05.006.2016242152502.hdf", 
               "MOD11A2.A2016201.h11v05.006.2016242234243.hdf", 
               "MOD11A2.A2016289.h11v05.006.2016302010943.hdf"]

for f in MODIS_files:
    
    #Extract from HDF
    day_LST = f.rstrip(".hdf") + "_dayLST.tif"
    arcpy.ExtractSubDataset_management(f, day_LST, subdataset_index=0)
    night_LST = f.rstrip(".hdf") + "_nightLST.tif"
    arcpy.ExtractSubDataset_management(f, night_LST, subdataset_index=4)
    
    #Scale
    day_LST_sc = arcpy.Raster(day_LST) * scale
    night_LST_sc = arcpy.Raster(night_LST) * scale
    
    #Perform subtraction
    diff = day_LST_sc - night_LST_sc
    diff.save(f.rstrip(".hdf")+"_diff.tif")
    
    #Apply conditional
    diff_thres = arcpy.sa.Con(diff > thres, 1, 0)
    diff_thres.save(f.rstrip(".hdf")+"_thres.tif")
    
    #Resample
    res = f.rstrip(".hdf") + "_res.tif"
    arcpy.Resample_management(diff_thres, res, cell_size=out_cellsize)
    
arcpy.CheckInExtension("spatial")

arcpy.env.workspace = None
arcpy.env.overwriteOutput = None
arcpy.env.mask = None

Inspect the output. If everything worked properly, you should only see data over the scenic Commonwealth of Virginia

__Very important note:__ The nature of environment settings is that they apply to your _entire processing environment_, which means they will stick around for future processes you run in the same session of Python. Old environment settings can lead to mysterious errors, confusion, and general gnashing of teeth. I highly recommend that you clear out your environment settings at the end of your script by setting them to `None`, as I have done above. Even better, clear them immediately after they are no longer needed, in case your script crashes before it completes.

### Editing locks

---

Whenever a program is using a file, it creates a lock that prevents other programs from editing the same file. Python is no different. In general, these should disappear after a script is finished executing. For reasons that are not entirely clear to me, ArcPy often retains locks on ArcPy objects created by a script even said script has completed. This will create problems if you then try to edit the file that object is pointing to in a different program. Therefore, it is also good practice to clear variables that point to ArcPy objects after they are no longer necessary. To accomplish that in our script, add the following to the end:

In [ ]:
day_LST_sc = None
night_LST_sc = None
diff = None
diff_thres = None

On the other side of the same coin, Python will not be able to edit a file that is locked by another application. In the context of geoprocessing, this can happen because you ran your code, viewed the output in ArcMap, and tried to rerun your code without removing said output from ArcMap. Some of you may have already experienced this phenomenon during this lesson. If you're me, this happens _all the time_...